Multiple Linear (Auto)Regression


For this exercise, we shall try to forecast store sales from
the Rossman dataset. Particularly, you are required to build multiple linear regression models to forecast
sales for the next 42 days.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df_train = pd.read_csv('Downloads/train.csv') #Read the required csv file and store it
df_store = pd.read_csv('Downloads/store.csv')

print(df_train)
store_count = df_train["Store"].value_counts()
print(store_count)

#Find Stores with recorded sales not equal to 942 and store its indices
store_numbers = store_count.index[store_count.values != 942] 

print(store_numbers)

index = df_train.index[df_train["Store"].isin(store_numbers)] 
print(index)

#Remove the stores with sales less than 942 and store it in new data frame
df_new = df_train.drop(index)


print(df_new)

#The number of stores having 942 sales are printed
print("Number of stores that have sales recorded for 942 days: ", df_new['Store'].nunique())



         Store  DayOfWeek        Date  Sales  Customers  Open  Promo  \
0            1          5  2015-07-31   5263        555     1      1   
1            2          5  2015-07-31   6064        625     1      1   
2            3          5  2015-07-31   8314        821     1      1   
3            4          5  2015-07-31  13995       1498     1      1   
4            5          5  2015-07-31   4822        559     1      1   
...        ...        ...         ...    ...        ...   ...    ...   
1017204   1111          2  2013-01-01      0          0     0      0   
1017205   1112          2  2013-01-01      0          0     0      0   
1017206   1113          2  2013-01-01      0          0     0      0   
1017207   1114          2  2013-01-01      0          0     0      0   
1017208   1115          2  2013-01-01      0          0     0      0   

        StateHoliday  SchoolHoliday  
0                  0              1  
1                  0              1  
2                  0 

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
sorted_df = df_new.sort_values(by=["Store"]) # Sort the dataframe by store number

req_df = sorted_df[["Store","Date","Sales"]] #Remove unwanted features

#Pivot the dataframe to obtain frame having the stores as rows and each daily sale as the column
pivot_df = req_df.pivot(index="Store",columns="Date",values="Sales")

print(pivot_df)

data = pivot_df.to_numpy()

print(data)




Date   2013-01-01  2013-01-02  2013-01-03  2013-01-04  2013-01-05  2013-01-06  \
Store                                                                           
1               0        5530        4327        4486        4997           0   
2               0        4422        4159        4484        2342           0   
3               0        6823        5902        6069        4523           0   
4               0        9941        8247        8290       10338           0   
5               0        4253        3465        4456        1590           0   
...           ...         ...         ...         ...         ...         ...   
1111            0        5097        4579        4640        3325           0   
1112            0       10797        8716        9788        9513           0   
1113            0        6218        5563        5524        5194           0   
1114            0       20642       18463       18371       18856           0   
1115            0        369

In [3]:
#Create a data matrix of the shape (#_of_stores, 942) for the daily sales record of these stores

data_matrix = np.asmatrix(data)
print("Shape of the data matrix:",data_matrix.shape)

Shape of the data matrix: (934, 942)


Use the first 800 stores in this data matrix for training and the rest for testing. Also split the
sales data into 2 parts, the 1st part contains the information about the first 900 days of sales
(these would be the features) and the 2nd contains the information about the last 42 days of
sales (these would be the targets). The resultant matrices should be of the dimensions:
𝑋𝑡𝑟𝑎𝑖𝑛 = (800, 900)
𝑋𝑡𝑒𝑠𝑡 = (#𝑟𝑒𝑚𝑎𝑖𝑛𝑖𝑛𝑔 𝑠𝑡𝑜𝑟𝑒𝑠, 900)
𝑌𝑡𝑟𝑎𝑖𝑛 = (800, 42)
𝑌𝑡𝑒𝑠𝑡 = (#𝑟𝑒𝑚𝑎𝑖𝑛𝑖𝑛𝑔 𝑠𝑡𝑜𝑟𝑒𝑠, 42)


In [4]:

X_train = data_matrix[:800,:900]
X_train.shape
X_train=np.hstack((np.ones((800,1)),X_train)) #For bias

In [5]:
Y_train = data_matrix[:800,900:942]
Y_train.shape

(800, 42)

In [6]:
X_test = data_matrix[800:,:900]

X_test.shape
X_test=np.hstack((np.ones((X_test.shape[0],1)),X_test)) #For bias
X_test.shape

(134, 901)

In [7]:
Y_test = data_matrix[800:,900:]
Y_test.shape

(134, 42)

We need to build multiple linear regression models to predict sales for each day,i.e, 42 models are required. We have X_train and Y_train

X_train. Beta = Y_train

We can solve for Beta using normal equations,

Beta = inv(X_train.T.X_train) * (X_train.Y_train) as done in lab 1

In [8]:
#Finding inverse using numpy

inverse = np.linalg.inv(X_train.T.dot(X_train)) 

In [9]:
B = X_train.T.dot(Y_train)

In [10]:
#Finding Beta matrix -  beta values are found for each day.

beta = inverse.dot(B)   # Beta is matrix of shape 901 * 42, i.e, each column corresponds to beta values for each store for a single day
beta.shape
print("beta values for 42 models:" ,beta)
print(beta.shape)


beta values for 42 models: [[ 1.36414273e+04  1.83300138e+04  1.69319810e+04 ...  1.97017202e+04
   2.01161470e+04  2.23595550e+04]
 [ 2.95260000e+04  5.06505000e+04  3.60605000e+04 ...  3.94895000e+04
   4.05930000e+04  4.59800000e+04]
 [-5.54162902e+00 -7.10656860e+00 -5.96194155e+00 ... -6.31723301e+00
  -6.51706919e+00 -8.00636473e+00]
 ...
 [-2.40211404e-01  1.23691279e-10  7.30315477e-02 ...  2.87205479e-01
   1.22761209e-01 -4.58099410e-01]
 [ 6.44860590e-02 -4.80213203e-10 -2.37147995e-01 ...  7.60697439e-02
   6.75380699e-01  7.23830475e-02]
 [ 1.91323700e-01  1.36788003e-09  2.90875849e-01 ... -4.30810227e-01
   1.77651261e-01  5.39095833e-01]]
(901, 42)


In [11]:
Y_predicted = np.matmul(X_train,beta) # predicting Y values from beta


In [12]:
print(Y_predicted.shape)

(800, 42)


In [13]:
beta1 = np.linalg.lstsq(X_train, Y_train, rcond=None)[0] 
beta1.shape

(901, 42)

Using these learned
parameters to make predictions for each day ahead. In total 42 days.

In [14]:
Y_predicted_test = np.matmul(X_test,beta) 
print(Y_predicted_test.shape)
print(Y_test.shape)

(134, 42)
(134, 42)


Function for finding Root Mean Square error and Mean absolute Error

In [15]:
def Error(Y_predicted_test,Y_test):
    Rmse = np.sqrt(np.mean(np.square((Y_predicted_test - Y_test))))
    print("Overall Rmse: ",Rmse)
    Mae = np.mean(np.abs(Y_predicted_test - Y_test))
    print("Overall MAE: ", Mae)


In [16]:
#To find Rmse of sales for each day (daily sales)
for i in range(42):
    sum =0
    sum1 =0
    for j in range(134):
        sum = sum + Y_predicted_test[j,i]
        sum1 = sum1 + Y_test[j,i]
    print("Rmse of sales in",i+1, "th day:",np.sqrt((np.square(sum - sum1))/134))
        

Rmse of sales in 1 th day: 69586741.50004596
Rmse of sales in 2 th day: 93475550.2265032
Rmse of sales in 3 th day: 84248569.91801976
Rmse of sales in 4 th day: 79053458.36100636
Rmse of sales in 5 th day: 78067877.57178217
Rmse of sales in 6 th day: 78020988.64852072
Rmse of sales in 7 th day: 84330375.17587627
Rmse of sales in 8 th day: 72670948.40695646
Rmse of sales in 9 th day: 96251590.71195032
Rmse of sales in 10 th day: 112352734.74793494
Rmse of sales in 11 th day: 114114523.7675407
Rmse of sales in 12 th day: 104924470.27767235
Rmse of sales in 13 th day: 102334947.00499521
Rmse of sales in 14 th day: 103766948.48011333
Rmse of sales in 15 th day: 73654481.5533537
Rmse of sales in 16 th day: 97930180.91676609
Rmse of sales in 17 th day: 91165350.02104656
Rmse of sales in 18 th day: 81782468.3314204
Rmse of sales in 19 th day: 85276486.69022481
Rmse of sales in 20 th day: 84048844.30382757
Rmse of sales in 21 th day: 87123679.14382777
Rmse of sales in 22 th day: 73454938.31561

In [17]:
Error(Y_predicted_test,Y_test) # Rmse and Mae of overall sales

Overall Rmse:  55435358.28851152
Overall MAE:  7878559.536782837


In [18]:
#Mae of sales for each day(daily sales)
for i in range(42):
    sum =0
    sum1 =0
    for j in range(134):
        sum = sum + Y_predicted_test[j,i]
        sum1 = sum1 + Y_test[j,i]
    print("Mae of sales in",i+1, "th day:",np.abs(sum - sum1)/134)
        

Mae of sales in 1 th day: 6011378.88209818
Mae of sales in 2 th day: 8075057.640452066
Mae of sales in 3 th day: 7277967.945256086
Mae of sales in 4 th day: 6829178.661108418
Mae of sales in 5 th day: 6744037.448641384
Mae of sales in 6 th day: 6739986.862609876
Mae of sales in 7 th day: 7285034.843186964
Mae of sales in 8 th day: 6277813.7785821725
Mae of sales in 9 th day: 8314871.012803398
Mae of sales in 10 th day: 9705798.007645871
Mae of sales in 11 th day: 9857993.398303209
Mae of sales in 12 th day: 9064093.694373103
Mae of sales in 13 th day: 8840392.955115715
Mae of sales in 14 th day: 8964099.04108976
Mae of sales in 15 th day: 6362778.101650699
Mae of sales in 16 th day: 8459879.120546447
Mae of sales in 17 th day: 7875486.739025511
Mae of sales in 18 th day: 7064929.215762159
Mae of sales in 19 th day: 7366766.429619432
Mae of sales in 20 th day: 7260714.280068038
Mae of sales in 21 th day: 7526339.553283407
Mae of sales in 22 th day: 6345540.191388883
Mae of sales in 23 t

Repeating last sale value -  Y is predicted using the last recorded sales value of each store
and repeat it for the next 42 days.

In [19]:
new_y = np.zeros((134,42))
for i in range(134):
    Last_Sale = Y_test[i,41] #getting last sale value
    for j in range(42):
        new_y[i,j] = Last_Sale # getting Y
print(new_y.shape)
print(new_y)

(134, 42)
[[10708. 10708. 10708. ... 10708. 10708. 10708.]
 [ 9081.  9081.  9081. ...  9081.  9081.  9081.]
 [ 8926.  8926.  8926. ...  8926.  8926.  8926.]
 ...
 [ 7289.  7289.  7289. ...  7289.  7289.  7289.]
 [27508. 27508. 27508. ... 27508. 27508. 27508.]
 [ 8680.  8680.  8680. ...  8680.  8680.  8680.]]


In [20]:
Error(new_y,Y_test) # errors btw actual y and newly predicted Y

Overall Rmse:  4646.157673062403
Overall MAE:  3532.9797441364603


Repeating Mean value per store - Repeat the mean of sales for the sales horizon

In [21]:
matrix = data_matrix[800:,] # Finding mean sales of each store
matrix.shape
mean = np.zeros((134))
for i in range(134):
    sum = 0
    for j in range(942):
        sum = sum + matrix[i,j]
    mean[i] = sum/942

print(mean.shape)



(134,)


In [22]:
new_y = np.zeros((134,42))
for i in range(134):
    mean_value = mean[i]
    for j in range(42):
        new_y[i,j] = mean_value # finding new y
        
print(new_y.shape)
print(new_y)

(134, 42)
[[ 5868.71019108  5868.71019108  5868.71019108 ...  5868.71019108
   5868.71019108  5868.71019108]
 [ 5574.45010616  5574.45010616  5574.45010616 ...  5574.45010616
   5574.45010616  5574.45010616]
 [ 5393.40552017  5393.40552017  5393.40552017 ...  5393.40552017
   5393.40552017  5393.40552017]
 ...
 [ 5516.18046709  5516.18046709  5516.18046709 ...  5516.18046709
   5516.18046709  5516.18046709]
 [17200.19639066 17200.19639066 17200.19639066 ... 17200.19639066
  17200.19639066 17200.19639066]
 [ 5225.29617834  5225.29617834  5225.29617834 ...  5225.29617834
   5225.29617834  5225.29617834]]


In [23]:
Error(new_y,Y_test) # errors btw actual y and newly predicted Y

Overall Rmse:  3120.2216810710515
Overall MAE:  2214.525431682956


Repeating Mean Value per store per weekday -  For each of the 42 days ahead get
their predictions as a mean of all sales recorded for that day of week in the past. For
e.g., the prediction of Monday ahead should be the mean of all sales for this
particular store on all previous Mondays.

In [24]:
matrix = data_matrix[800:,]
matrix.shape
mean = np.zeros((134,7))
for i in range(134):
    sum1=sum2=sum3=sum4=sum5=sum6=sum7 = 0
    for j in range(0,942,7):
        sum1 = sum1 + matrix[i,j]        #Finding sum of sales of each day of the week
        sum2 = sum2 + matrix[i,j+1]
        sum3 = sum3 + matrix[i,j+2]
        sum4 = sum4 + matrix[i,j+3]
        if(j!=938):
            sum5 = sum5 + matrix[i,j+4]
            sum6 = sum6 + matrix[i,j+5]
            sum7 = sum7 + matrix[i,j+6]
    mean[i,0] = sum1/135  # Finding mean of sales for each day of week
    mean[i,1] = sum2/135
    mean[i,2] = sum3/135
    mean[i,3] = sum4/135
    mean[i,4] = sum5/134
    mean[i,5] = sum6/134
    mean[i,6] = sum7/134
    
print(mean)
print(mean.shape)



[[ 7642.35555556  7056.94814815  6680.08148148  6928.14074074
   4338.78358209     0.          8398.59701493]
 [ 6646.94074074  6352.9037037   5810.25925926  6478.6962963
   6198.93283582     0.          7511.09701493]
 [ 6443.77037037  5982.83703704  5638.55555556  6694.53333333
   5998.58955224     0.          6971.7761194 ]
 [ 7184.58518519  5846.51851852  5597.44444444  6402.74814815
   5105.04477612     0.          7923.20895522]
 [13065.74074074 10300.8        10856.74074074  9599.21481481
   4189.63432836     0.         14238.49253731]
 [ 9432.2         9166.8         8576.02222222  8887.57037037
   9287.89552239     0.         10609.48507463]
 [ 8662.46666667  8671.44444444  7372.71111111  7831.59259259
   7026.89552239     0.          9644.30597015]
 [ 5066.22962963  4738.58518519  4293.14814815  4759.15555556
   3631.82835821     0.          5379.90298507]
 [ 8803.55555556  8013.27407407  8688.46666667  8470.75555556
   6152.68656716     0.          9852.58208955]
 [ 7632.029

   6773.3880597      0.          6704.2238806 ]]
(134, 7)


In [25]:
new_y = np.zeros((134,42))
for i in range(134):
    for j in range(0,42,7):
        new_y[i,j] = mean[i,0]
        new_y[i,j+1] = mean[i,1]
        new_y[i,j+2] = mean[i,2]
        new_y[i,j+3] = mean[i,3]
        new_y[i,j+4] = mean[i,4]
        new_y[i,j+5] = mean[i,5]
        new_y[i,j+6] = mean[i,6]
        
print(new_y.shape)
print(new_y)

(134, 42)
[[ 7642.35555556  7056.94814815  6680.08148148 ...  4338.78358209
      0.          8398.59701493]
 [ 6646.94074074  6352.9037037   5810.25925926 ...  6198.93283582
      0.          7511.09701493]
 [ 6443.77037037  5982.83703704  5638.55555556 ...  5998.58955224
      0.          6971.7761194 ]
 ...
 [ 6512.56296296  6308.62222222  6570.96296296 ...  5893.57462687
      0.          7006.65671642]
 [19855.24444444 19082.67407407 18593.32592593 ... 21935.5
      0.         20900.17164179]
 [ 5811.53333333  5666.15555556  5396.60740741 ...  6773.3880597
      0.          6704.2238806 ]]


In [26]:
Error(new_y,Y_test) # errors btw actual y and newly predicted Y

Overall Rmse:  4289.21028359613
Overall MAE:  3236.6643648613126


From evaluating the various RMSEs and MAEs we find that the highest error occurred when sales were predicted using linear regression. Hence regression is not a good choice for this case.

Reference

https://pandas.pydata.org/pandas-docs/version/0.25.1/reference/api/pandas.DataFrame.to_numpy.html#pandas.DataFrame.to_numpy
    https://pandas.pydata.org/pandas-docs/dev/user_guide/reshaping.html